In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Glucocorticoid_Sensitivity"
cohort = "GSE65645"

# Input paths
in_trait_dir = "../../input/GEO/Glucocorticoid_Sensitivity"
in_cohort_dir = "../../input/GEO/Glucocorticoid_Sensitivity/GSE65645"

# Output paths
out_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/GSE65645.csv"
out_gene_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE65645.csv"
out_clinical_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/clinical_data/GSE65645.csv"
json_path = "../../output/preprocess/Glucocorticoid_Sensitivity/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import numpy as np
import json
from typing import Optional, Callable, Dict, Any, List

# 1. Gene Expression Data Availability
# Looking at the background info, this is an Agilent microarray with lncRNA expression data
# It's likely to contain gene expression data, though focused on lncRNAs
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Glucocorticoid_Sensitivity)
# Looking at the background info, this study involves responsiveness to prednisolone/prednisone (glucocorticoids)
# From the sample characteristics dictionary, we can use the 'translocation' information as it relates to 
# glucocorticoid sensitivity in B-ALL
trait_row = 1  # corresponds to the translocation types

# For age and gender
# These are not available in the sample characteristics dictionary
age_row = None
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value: str) -> Optional[int]:
    """
    Convert translocation types to binary for glucocorticoid sensitivity.
    
    Based on the background information, MLL translocations are associated with 
    poorer response to glucocorticoids, so we'll use this as the binary indicator.
    0 = TEL_AML1 or E2A_PBX1 (better glucocorticoid response)
    1 = MLL (worse glucocorticoid response)
    """
    if not value or ':' not in value:
        return None
    
    translocation = value.split(':', 1)[1].strip()
    
    if translocation == 'MLL':
        return 1  # Less sensitive to glucocorticoids
    elif translocation in ['TEL_AML1', 'E2A_PBX1']:
        return 0  # More sensitive to glucocorticoids
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """
    Placeholder function for age conversion (not used as age data is not available).
    """
    return None

def convert_gender(value: str) -> Optional[int]:
    """
    Placeholder function for gender conversion (not used as gender data is not available).
    """
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Save initial metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Assuming the clinical data is stored somewhere and accessible
    # We'll create a properly formatted DataFrame based on the sample characteristics
    
    # First, create columns for samples
    sample_chars = {0: ['sample_type: bone marrow'], 1: ['translocation: TEL_AML1', 'translocation: E2A_PBX1', 'translocation: MLL']}
    
    # The format expected by geo_select_clinical_features seems to be:
    # - Rows represent different characteristics (like sample_type, translocation)
    # - Columns represent different samples
    
    # Create a DataFrame with sample names as columns
    num_samples = max(len(values) for values in sample_chars.values())
    sample_columns = [f'Sample_{i+1}' for i in range(num_samples)]
    
    clinical_data = pd.DataFrame(index=sample_chars.keys(), columns=sample_columns)
    
    # Fill in the data
    for row_idx, values in sample_chars.items():
        for sample_idx, value in enumerate(values):
            if sample_idx < len(sample_columns):
                clinical_data.iloc[row_idx, sample_idx] = value
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the DataFrame
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 4: Gene Identifier Review

In [ ]:
# Based on the gene identifiers shown in the sample data, these appear to be Agilent microarray
# probe IDs (starting with A_19_P) mixed with control probes. These are not standard human
# gene symbols and will require mapping to convert to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Determine which keys to use for mapping
# From the outputs, we can see:
# - The gene expression data uses 'ID' as identifiers
# - In the annotation dataframe, 'ID' corresponds to probe IDs and 'GENE_SYMBOL' contains gene symbols

# 2. Get gene mapping dataframe
gene_mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# 3. Apply gene mapping to convert probe-level measurements to gene expression
# First, let's check if there are any rows in the mapping dataframe
print(f"Number of rows in mapping dataframe: {len(gene_mapping_df)}")
print(f"Sample of gene mapping data (first 5 rows):")
print(gene_mapping_df.head())

# Apply the mapping
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)

# Check the resulting gene expression data
print(f"\nNumber of genes after mapping: {len(gene_data)}")
print(f"First few genes:")
print(gene_data.index[:10])

# Preview the gene expression data
print("\nPreview of gene expression data:")
gene_data_preview = preview_df(gene_data)
print(gene_data_preview)

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 2. Re-load the clinical data from the SOFT file to extract sample characteristics
print("\nExtracting clinical data from SOFT file...")
sample_info = {}

# Read the SOFT file to extract sample information
with gzip.open(soft_file, 'rt') as f:
    current_sample = None
    for line in f:
        # Detect sample sections
        if line.startswith('!Sample_geo_accession'):
            sample_id = line.split('=')[1].strip()
            current_sample = sample_id
            sample_info[current_sample] = {}
        
        # Extract translocation information
        if current_sample and 'translocation' in line.lower():
            if 'mll' in line.lower() or 'mil' in line.lower():
                sample_info[current_sample]['translocation'] = 'MLL'
            elif 'tel' in line.lower() and 'aml' in line.lower():
                sample_info[current_sample]['translocation'] = 'TEL_AML1'
            elif 'e2a' in line.lower() and 'pbx' in line.lower():
                sample_info[current_sample]['translocation'] = 'E2A_PBX1'

print(f"Found information for {len(sample_info)} samples")

# Map samples to trait values based on translocation type
clinical_data = {}
genetic_sample_ids = normalized_gene_data.columns.tolist()

for sample_id in genetic_sample_ids:
    if sample_id in sample_info and 'translocation' in sample_info[sample_id]:
        translocation = sample_info[sample_id]['translocation']
        if translocation == 'MLL':
            clinical_data[sample_id] = 1  # Less sensitive to glucocorticoids
        elif translocation in ['TEL_AML1', 'E2A_PBX1']:
            clinical_data[sample_id] = 0  # More sensitive to glucocorticoids
    else:
        clinical_data[sample_id] = None

# Create clinical dataframe
clinical_df = pd.DataFrame({trait: clinical_data}, index=genetic_sample_ids)
print(f"Clinical data shape: {clinical_df.shape}")
print("Clinical data preview:")
print(clinical_df.head())
print(f"Samples with valid trait values: {clinical_df[trait].notna().sum()}")

# Save updated clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Updated clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = pd.concat([clinical_df, normalized_gene_data.T], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
gene_cols = [col for col in linked_data.columns if col != trait]
if gene_cols:
    missing_genes_pct = linked_data[gene_cols].isna().mean()
    genes_with_high_missing = sum(missing_genes_pct > 0.2)
    print(f"  Genes with >20% missing: {genes_with_high_missing}")
    
    if len(linked_data) > 0:  # Ensure we have samples before checking
        missing_per_sample = linked_data[gene_cols].isna().mean(axis=1)
        samples_with_high_missing = sum(missing_per_sample > 0.05)
        print(f"  Samples with >5% missing genes: {samples_with_high_missing}")

# Handle missing values
cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
if len(cleaned_data) > 0:
    trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
else:
    trait_biased = True
    print("Dataset is empty after handling missing values.")

# 6. Final validation and save
note = "Dataset contains gene expression data from B-ALL patients with different translocations. "
note += "Translocation type is used as a proxy for glucocorticoid sensitivity: MLL translocations have poorer "
note += "response to glucocorticoids compared to TEL_AML1 or E2A_PBX1 translocations. No demographic features available."

is_gene_available = len(normalized_gene_data) > 0
is_trait_available = len(clinical_df) > 0 and clinical_df[trait].notna().sum() > 0

is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=trait_biased, 
    df=cleaned_data,
    note=note
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")